# Importação das Bibliotecas e do DataFrame

In [ ]:
# Importando as Bibliotecas
import pandas as pd
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# DataFrame do Campeonato Brasileiro
df = pd.read_excel("https://www.football-data.co.uk/new/BRA.xlsx")

In [ ]:
df = df[['Season','Date','Home','Away','HG','AG','Res','PH','PD','PA']]
df.columns = ['Season','Date','Home','Away','Goals_H','Goals_A','Result','Odds_H','Odds_D','Odds_A']

In [ ]:
# Filtrando as Temporadas
flt = (df.Season == 2021) | (df.Season == 2022) 
df = df[flt]
# Resetando o Index
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df = df.dropna()

# Variáveis Secundárias

In [ ]:
def Prob_Home():
    P_H = 1/df.Odds_H
    return P_H

def Prob_Draw():
    P_D = 1/df.Odds_D
    return P_D

def Prob_Away():
    P_A = 1/df.Odds_A
    return P_A

In [ ]:
df['p_H'] = Prob_Home()
df['p_D'] = Prob_Draw()
df['p_A'] = Prob_Away()

In [ ]:
def Valor_do_Gol_H():
    VG_H = df.Goals_H * df.p_A
    return VG_H 

def Valor_do_Gol_A():
    VG_A = df.Goals_A * df.p_H
    return VG_A 

In [ ]:
df['Valor_do_Gol_H'] = Valor_do_Gol_H()
df['Valor_do_Gol_A'] = Valor_do_Gol_A()

In [ ]:
df['Media_Valor_do_Gol_H'] = df.groupby('Home')['Valor_do_Gol_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Valor_do_Gol_A'] = df.groupby('Away')['Valor_do_Gol_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

In [ ]:
def Custo_do_Gol_H():
    CG_H = df.p_H / df.Goals_H
    return CG_H 

def Custo_do_Gol_A():
    CG_A = df.p_A / df.Goals_A
    return CG_A 

In [ ]:
df['Custo_do_Gol_H'] = Custo_do_Gol_H()
df['Custo_do_Gol_A'] = Custo_do_Gol_A()

In [ ]:
df.replace(np.inf, 1, inplace=True)

In [ ]:
df['Media_Custo_do_Gol_H'] = df.groupby('Home')['Custo_do_Gol_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Custo_do_Gol_A'] = df.groupby('Away')['Custo_do_Gol_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

In [ ]:
team = "Sao Paulo"
df1 = df.groupby(['Home']).get_group(team)
df2 = df1[['Home','Away','Goals_H','Goals_A','Result','Media_Valor_do_Gol_H','Media_Valor_do_Gol_A','Media_Custo_do_Gol_H','Media_Custo_do_Gol_A']]
df2.tail(5)